In [ ]:
import os
import time
import random
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.utils import _pair, _quadruple
from sklearn.model_selection import train_test_split

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

class CustomDataset(Dataset):
    """Stores data on CPU and lets DataLoader move to device.
    FC: (N, H, W, C_in)
    DTO: (N, H, W, 1)
    com: (N,) scalar compliance targets per sample
    """
    def __init__(self,FC,DTO,com):
        self.x_data = torch.cuda.FloatTensor(FC)
        self.x_data = self.x_data.permute(0,3,1,2) #

        self.y_data = torch.cuda.FloatTensor(DTO)
        self.y_data = self.y_data.permute(0,3,1,2)

        self.z_data = torch.cuda.FloatTensor(com)
        
        self.len = self.y_data.shape[0]
    def __len__(self):
        return self.len
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index], self.z_data[index]

# proposed U-Net based encoder decoder network for training
class CNN_top(nn.Module):
    def __init__(self):
        super(CNN_top, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(4,32,2),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32,32,3,padding="same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            )
        self.layer2 = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding="same"),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(64,64,3,padding="same"),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            )
        self.layer3 = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding="same"),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128,128,3,padding="same"),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            )
        self.layer4 = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(128,256,3,padding="same"),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256,256,3,padding="same"),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            )
        self.layer5 = nn.Sequential(
            nn.Conv2d(384,128,3,padding="same"),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128,128,3,padding="same"),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            )
        self.layer6 = nn.Sequential(
            nn.Conv2d(192,64,3,padding="same"),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(64,64,3,padding="same"),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            )
        self.layer7 = nn.Sequential(
            nn.Conv2d(96,32,3,padding="same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32,32,3,padding="same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            )
        self.layer8 =  nn.Sequential(
             nn.Conv2d(32,1,3,padding="same"),
             nn.Sigmoid(),
            )
        self.conv1 = nn.Sequential(
            nn.Conv2d(384,384,1,1,padding=0),
            nn.BatchNorm2d(384),
            nn.ReLU(),
            )
        self.conv2 = nn.Sequential(
            nn.Conv2d(192,192,1,1,padding=0),
            nn.BatchNorm2d(192),
            nn.ReLU(),
            )
        self.conv3 = nn.Sequential(
            nn.Conv2d(96,96,1,1,padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            )
        self.avg = nn.AvgPool2d(3,1,1)

    def forward(self, x):
        x1 = self.layer1(x)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        
        x = self.layer4(x3)
        x = torch.cat([x,x3],dim=1) # concatenate layer 256+128 =384
        
        # topology CNN filter 1
        fil1 = x
        x = self.conv1(fil1)
        ori1 = x
        x = self.avg (fil1)
        x = self.conv1(x)
        x = torch.add(x,ori1)
        
        x = self.layer5(x)
        x = torch.cat([x,x2],dim=1) # concatenate layer 128+64=192
        
        # topology CNN filter 2 (TO CNN filter 1 and 2 are most effective)
        fil2 = x
        x = self.conv2(fil2)
        ori2 = x
        x = self.avg (fil2)
        x = self.conv2(x)
        x = torch.add(x,ori2)

        x = self.layer6(x)
        x = torch.cat([x,x1],dim=1) # concatenate layer 64+32=96 
        x = self.layer7(x)
        x = self.layer8(x)
        eps = 1e-6
        return x * (1 - eps) + eps # modified output layer for preventing numerical instability

# constraint loss with MAE loss
def con_loss(pre,real):
    return torch.mean(abs(torch.mean(pre)-torch.mean(real))) 

# compliance loss with FEM and one random culculation per batch
def comploss(DTO,FC,comp,penal=3,Emax=1,Emin=1e-9,nu=0.3): 
    # final version of compliance loss 
    # torch.set_default_dtype(torch.float64) (use)
    
    # input data
    n = min(DTO.shape[0],FC.shape[0],comp.shape[0]) 
    m = random.randint(0,n)-1   # one random comploss calculation per batch
    
    # TO structure
    DTO_x = DTO[m,0,:,:].T
    nelx, nely = DTO_x.shape
    x = torch.flatten(DTO_x)
    
    # Force maaping
    Fx_map = FC[m, 0, :, :].T 
    Fy_map = FC[m, 1, :, :].T

    Fx_vec = Fx_map.flatten()
    Fy_vec = Fy_map.flatten()
    
    fx = torch.where(Fx_vec!=0)[0]*2
    fy = torch.where(Fy_vec!=0)[0]*2+1
    F_idx = torch.hstack((fx,fy))
    
    Bx = Fx_vec[torch.where(Fx_vec!=0)]
    By = Fy_vec[torch.where(Fy_vec!=0)]
    B = torch.hstack((Bx,By))
    
    C = F_idx.numel()
    
    BCM = FC[m, 3, :, :].T
    l=len(torch.where(BCM==1)[0])
    BC_c=torch.where(BCM==1)
    BC_a=torch.zeros(2*l,dtype=int)

    for i in range(l):
        BC_a[2*i]=2*(BC_c[0][i]+1)*(BC_c[1][i]+1)-2
        BC_a[2*i+1]=2*(BC_c[0][i]+1)*(BC_c[1][i]+1)-1

    # K matrix setting
    ndof = 2*(nelx+1)*(nely+1)
    edofMat = torch.zeros((nelx*nely,8),dtype=int)
    nodenrs = torch.transpose(torch.reshape(torch.arange((1+nelx)*(1+nely)),(1+nelx,1+nely)),1,0)
    edofVec = torch.reshape(torch.transpose((2*nodenrs[:-1,:-1]+1),1,0),(nely*nelx,1))
    edofMat = torch.tile(edofVec,(1,8))+torch.tile(torch.tensor([1, 2, 2*nely+3, 2*nely+4, 2*nely+1 ,2*nely+2, -1, 0]),(nelx*nely,1))
    iK = torch.flatten(torch.kron(edofMat,torch.ones((8,1)))).cuda()
    jK = torch.flatten(torch.kron(edofMat,torch.ones((1,8)))).cuda()

    A11 = torch.transpose(torch.reshape(torch.tensor([12,  3, -6, -3,  3, 12,  3,  0, -6,  3, 12, -3, -3,  0, -3, 12]),(4,4)),1,0)
    A12 = torch.transpose(torch.reshape(torch.tensor([-6, -3,  0,  3, -3, -6, -3, -6,  0, -3, -6,  3,  3, -6,  3, -6]),(4,4)),1,0)
    B11 = torch.transpose(torch.reshape(torch.tensor([-4,  3, -2,  9,  3, -4, -9,  4, -2, -9, -4, -3,  9,  4, -3, -4]),(4,4)),1,0)
    B12 = torch.transpose(torch.reshape(torch.tensor([ 2, -3,  4, -9, -3,  2,  9, -2,  4,  9,  2,  3, -9, -2,  3,  2]),(4,4)),1,0)
    KE = 1/(1-nu**2)/24*(torch.vstack((torch.hstack((A11,A12)),torch.hstack((A12.T, A11))))
                        +nu*torch.vstack((torch.hstack((B11,B12)),torch.hstack((B12.T, B11))))).cuda()

    dofs = torch.arange(2*(nelx+1)*(nely+1))
    fixed = BC_a
    combined = torch.cat((dofs, fixed))
    uniques, counts = combined.unique(return_counts=True)
    free = uniques[counts == 1]

    f = torch.zeros((ndof,C)).cuda()
    u = torch.zeros((ndof,C)).cuda()

    for j in range(C):
          f[F_idx[j],j] = B[j]
    sK = (torch.unsqueeze(KE.flatten(),0).T*(Emin+(x)**penal*(Emax-Emin))).T.flatten()
    K = torch.sparse_coo_tensor(torch.vstack((iK,jK)),sK,[ndof,ndof]).to_dense()
    
    # Remove constrained dofs from matrix
    K = (K[free,:][:,free])
    # Solve system 
    if C==1:
        u[free,0]=torch.linalg.solve(K,f[free,0])
    else:
        u[free,:]=torch.linalg.solve(K,f[free,:])

    # Objective and sensitivity
    ce=torch.ones(nely*nelx).cuda()
    obj=0
    KE = KE.to(torch.float64)
    
    for i in range(C):
        ui = u[:,i]
        mod_ui = ui[edofMat].reshape(nelx*nely,8).to(torch.float64)
        ce = (torch.matmul(mod_ui,KE) * mod_ui).sum(1)
        obj = obj+( (Emin+x**penal*(Emax-Emin))*ce ).sum()

    return torch.relu(obj/comp[m]-1)

# loss polt
def plot_loss(trn_loss_list,val_loss_list,title='Total Loss',save='Pytorch Total Loss & test.png'):
    plt.figure(figsize=(20,20))
    plt.plot(trn_loss_list)
    plt.plot(val_loss_list)
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training data', 'Validation data'], loc=0)
    plt.rc('font', size=40)        
    plt.rc('axes', labelsize=50)   
    plt.rc('xtick', labelsize=40) 
    plt.rc('ytick', labelsize=40) 
    plt.rc('legend', fontsize=30)  
    plt.rc('figure', titlesize=200)
    plt.savefig(save)

def topNN(DTO_data_path, FC_data_path, com_data_path, save_path, num_epochs = 200, data_num = 2000,
          lamda1 = 5*10**(-15),lamda2 = 10**(-5),mini_batch_size=128):
    
    # data load
    DTO = np.load(DTO_data_path)[0:data_num,:,:,:]
    FC = np.load(FC_data_path)[0:data_num,:,:,:]
    com = np.load(com_data_path)[0:data_num]

    # data split 
    DTO_train, DTO_test, com_train, com_test, FC_train, FC_test = \
                train_test_split(DTO,com,FC, test_size=0.2, shuffle=True)
    DTO_train, DTO_val, com_train, com_val, FC_train, FC_val = \
                train_test_split(DTO_train,com_train,FC_train, test_size=0.2, shuffle=True)

    tr_dataset = CustomDataset(FC_train,DTO_train,com_train)
    val_dataset = CustomDataset(FC_val,DTO_val,com_val)
    test_dataset = CustomDataset(FC_test,DTO_test,com_test)
    
    tr_dataloader = DataLoader(tr_dataset, batch_size = mini_batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size = mini_batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size = 1, shuffle=True)

    cnn = CNN_top().to(device) # model 
    
    # define the main loss function
    loss_fun = nn.BCELoss()
    
    # define the optimizer
    optimizer = torch.optim.Adam(cnn.parameters(), 
                                 lr=0.001, betas=(0.9, 0.999))

    os.makedirs(save_path)
    os.chdir(save_path)
    
    f = open('torch_train_results.txt', 'w')
    print('\nlambda1: ',lamda1, file=f)
    print('\nlambda2: ',lamda2, file=f)
    print('\nbatch_size: ', mini_batch_size, file=f)

    trn_loss_list = []
    train_loss_list = []
    train_comploss_list = []
    train_conloss_list = []
    
    val_loss_list = []
    vali_loss_list = []
    vali_comploss_list = []
    vali_conloss_list = []
    
    trigger_times = 0
    patience = 10
    start_time = time.time()
    print("Start")

    for epoch in range(num_epochs):
        trn_loss = 0.0
        cnn.train()

        for i, samples in enumerate(tr_dataloader):
            x_train, y_train, z_train = samples
            y_train.requires_grad=True
            optimizer.zero_grad()
            output = cnn.forward(x_train)
            
            train_loss = loss_fun(output,y_train)
            train_comploss = comploss(output,x_train.detach(),z_train.detach())
            train_conloss = con_loss(output,y_train)
            
            loss = train_loss+lamda1*train_comploss+lamda2*train_conloss
                
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            val_loss = 0.0
            for j, val in enumerate(val_dataloader):
                x_val, y_val, z_val = val
                val_output = cnn(x_val)

                vali_loss = loss_fun(val_output,y_val)
                vali_comploss = comploss(val_output,x_val.detach(),z_val.detach())
                vali_conloss = con_loss(val_output,y_val)

                v_loss = vali_loss+lamda1*vali_comploss+lamda2*vali_conloss
                val_loss += v_loss

            print(("epoch: {}/{} \ntrain: | tot loss: {:.4f} | BCE loss: {:.4f} | comp loss: {:.4f} |"
                   +" cons loss: {:.4f} | \nvalid: | tot loss: {:.4f} | BCE loss: {:.4f} |"+" comp loss: {:.4f} | cons loss: {:.4f} |")
                  .format(epoch+1, num_epochs,
              loss.item(),
              train_loss.item(),
              train_comploss.item(),   # <- () 빠졌던 부분
              train_conloss.item(),
              v_loss.item(),
              vali_loss.item(),
              vali_comploss.item(),    # <- () 빠졌던 부분
              vali_conloss.item()))
            
            trn_loss_list.append(loss.item())
            train_loss_list.append(train_loss.item())
            train_comploss_list.append(train_comploss.item())
            train_conloss_list.append(train_conloss.item())
        
            val_loss_list.append(v_loss.item())
            vali_loss_list.append(vali_loss.item())
            vali_comploss_list.append(vali_comploss.item())
            vali_conloss_list.append(vali_conloss.item())

        # Early Stopping criterion patence
        if epoch>1:
            if val_loss_list[epoch]>val_loss_list[epoch-1]:
                trigger_times +=1
                if trigger_times >= patience:
                    print('Early stopping! {}'.format(epoch+1))
                    break
            else:
                trigger_times = 0
        torch.save(cnn, 'model'+str(epoch)+'.pt') 
    print("End")
    print("Time: {:.4f}sec".format((time.time() - start_time)))

    print('\nTotal iteration: {}/{}'.format(epoch+1, num_epochs), file=f)
    print('\nTraining time: {:.4f}sec'.format((time.time() - start_time)), file=f)
    
    plot_loss(trn_loss_list,val_loss_list,'Total Loss','Pytorch Total Loss & test1.png')
    plot_loss(train_loss_list,vali_loss_list,'BCE Loss','Pytorch BCE Loss & test.png')
    plot_loss(train_comploss_list,vali_comploss_list,'comp Loss','Pytorch comp Loss & test.png')
    plot_loss(train_conloss_list,vali_conloss_list,'con Loss','Pytorch con Loss & test.png')
    plot_loss(trn_loss_list,val_loss_list,'Total Loss','Pytorch Total Loss & test.png')

    test_loss_fun = nn.L1Loss() 

    cnn.eval()
    start_time = time.time()
    print("Start")
    
    test_loss = 0.0
    test_comploss = 0.0
    test_conloss = 0.0
    with torch.no_grad():
        for i, samples in enumerate(test_dataloader):
            x_test, y_test, z_test = samples
            test_output = cnn(x_test)
            test_loss += test_loss_fun(test_output,y_test)
            test_comploss += comploss(test_output, x_test.detach(), z_test.detach())
            test_conloss += con_loss(torch.mean(test_output),torch.mean(y_test))

            if i == 0: 
                test_out = test_output
                x_test_out = x_test
                y_test_out = y_test
                z_test_out = z_test
            else:
                test_out = torch.cat([test_out,test_output], dim=0)
                x_test_out = torch.cat([x_test_out,x_test], dim=0)
                y_test_out = torch.cat([y_test_out,y_test], dim=0)
                z_test_out = torch.cat([z_test_out,z_test], dim=0)

    print("End")
    print("Time: {:.4f}sec".format((time.time() - start_time)))
    print("test loss: {:.4f} | comp loss: {:.4f} | cons loss: {:.4f}"
          .format(test_loss/(i+1),test_comploss/(i+1),test_conloss/(i+1)))
    print("\noriginal test loss: {:.4f} | comp loss: {:.4f} | cons loss: {:.4f}"
          .format(test_loss/(i+1),test_comploss/(i+1),test_conloss/(i+1)), file=f)
    
    accuracy_test = abs(test_out - y_test_out)
    accuracy_test = accuracy_test.cpu().numpy()
    acc = len(accuracy_test[np.where(accuracy_test<0.5)])/test_out.cpu().numpy().size*100
    print(acc, file=f)

    save_test = test_out
    save_test = save_test.permute(0,2,3,1)
    save_test = save_test.detach().cpu().numpy()

    save_x_test = x_test_out
    save_x_test = save_x_test.permute(0,2,3,1)
    save_x_test = save_x_test.detach().cpu().numpy()

    save_y_test = y_test_out
    save_y_test = save_y_test.permute(0,2,3,1)
    save_y_test = save_y_test.detach().cpu().numpy()

    save_z_test = z_test_out
    save_z_test = save_z_test.detach().cpu().numpy()

    np.save('save_new_test.npy',save_test)
    np.save('save_x_new_test.npy',save_x_test)
    np.save('save_y_new_test.npy',save_y_test)
    np.save('save_z_new_test.npy',save_z_test)

    torch.save(cnn, 'model.pt')  # save the model 
    torch.save(cnn.state_dict(), 'model_state_dict.pt')  # save the state_dict of the model
    torch.save({
        'model': cnn.state_dict(),
        'optimizer': optimizer.state_dict()
    }, 'all.tar')  
    f.close()
    return test_loss/(i+1), test_comploss/(i+1), acc

In [ ]:
test_loss = []
test_comploss = []
test_acc = []

epoch = 200
data_num = 2000
batch_size = 32

DTO_data_path = 'newDTO_3232_cantilver_beam_20k.npy'
com_data_path = 'newcomp_3232_cantilver_beam_20k.npy'
FC_data_path = 'newFCv_3232_cantilver_beam_20k.npy'

save_path = 'C:\\Users\\user\\'
name = "2000data_32x32_new_additional_test"

a = topNN(DTO_data_path, FC_data_path, com_data_path,
          save_path+name,
          epoch,data_num, 10**(-5), 10**(-4), batch_size)

test_loss.append(a[0])
test_comploss.append(a[1])
test_acc.append(a[2])